# Jupyter - Day 16 - Section 002
# Lect 16: Subset Selection

In [1]:
# Everyone's favorite standard imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.linear_model import LinearRegression,LogisticRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

In [3]:
# First, we're going to do all the data loading we've had for a while for this data set
auto = pd.read_csv('../data/Auto.csv')# update path to where you downloaded the csv
auto = auto.replace('?', np.nan)
auto = auto.dropna()
auto.horsepower = auto.horsepower.astype('int')

#this shuffles my data set in advance so that i don't need to worry about it later 
auto = auto.sample(frac=1).reset_index(drop=True)


auto.head()


,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name
0,26.5,4,140.0,72,2565,13.6,76,1,ford pinto
1,34.4,4,98.0,65,2045,16.2,81,1,ford escort 4w
2,24.2,6,146.0,120,2930,13.8,81,3,datsun 810 maxima
3,21.0,6,155.0,107,2472,14.0,73,1,mercury capri v6
4,21.0,6,199.0,90,2648,15.0,70,1,amc gremlin


Let's try to run subset selection on the `auto` data set! We're going to use `cylinders`, `horsepower`, `weight`, and `acceleration` to predict `mpg`. 

In [4]:
inputvars = ['cylinders','horsepower','weight', 'acceleration']

The first tool we are going to use is the `itertools` package, which gives us a way to get subsets of whatever size we want using the `combinations` command.  

In [5]:
from itertools import combinations

The weird thing is it's an iterator, so if I just try to print out what I want, it's not helpful to me. 

In [6]:
combinations(inputvars,2)

But if I use it in a for loop it does what I want!

In [7]:
for x in combinations(inputvars,2):
    print(x)

('cylinders', 'horsepower')
('cylinders', 'weight')
('cylinders', 'acceleration')
('horsepower', 'weight')
('horsepower', 'acceleration')
('weight', 'acceleration')


Here's some code stolen from the last few days to run linear regression on a subset of the input variables and get the *training error*. Note that this has no train/test split.  

In [8]:
def myscore(df,listofvars, outputvar = 'mpg'):
    '''
    This code returns the training mean squared error of a linear regression model. No train test split or anything like that. 
    '''
    if listofvars == 'null model':
        # Predict the mean of the output variable for any input 
        yhat = np.full(auto.shape[0],np.average(auto.mpg))
        return mean_squared_error(auto.mpg, yhat)
    
    else:
        X = df[list(listofvars)]
        y = df[outputvar]
        
        #build linear regression model
        model = LinearRegression()
        
        # fit model
        model.fit(X,y)

        #predict y
        ypred = model.predict(X)

        #view mean absolute error
        return mean_squared_error(y,ypred)
    

myvars = ('cylinders', 'acceleration')
print(f"MSE for the (cylinders, acceleration) model: {myscore(auto,myvars)}")

print(f"MSE for the null model: {myscore(auto,'null model')}")

MSE for the (cylinders, acceleration) model: 23.942446650601354
MSE for the null model: 60.76273844231571



&#9989; **<font color=red>Do this:</font>** Modify the code below by  using the `myscore` function to get the training error for each possible model of size $k=2$.


In [ ]:
def getBestModel_df(df, k=2, outputvar = 'mpg'):
    '''
    This function will return the best model for a given number of input variables
    '''
    myvars = []
    myscores = []
    
    if k == 0:
        return pd.DataFrame({'Vars':'null model', 'Train Score':[myscore(auto,'null model')]})
    else:

        for Xs in combinations(inputvars,k):
            myvars.append(Xs) 
            myscores.append(None) #<--- Fix this None
                
        myResults = pd.DataFrame({'Vars':myvars, 'Train Score':myscores})
        return myResults

print("k = 2")
print(getBestModel_df(auto, k=2))
print("\n")
print("k = 0, so this is the null model")
print(getBestModel_df(auto, k=0))


&#9989; **<font color=red>Q:</font>** What is $M_2$? That is, what is the set of variables that give the lowest training error over all sets of size 2? Write a command to pull this automatically from the table above. 

*Hint: the command `DataFrame.idxmin()` will give you the row index of the minimum entry in a pandas series.*


In [ ]:
# Your code here 

def getBestModel(df, k=2, outputvar = 'mpg'):
    '''
    This function will return the best model for a given number of input variables
    '''
    myResults = getBestModel_df(df, k, outputvar)
    
    # Add code here to find the index of the minimum training score, return variable names
    M_k = None
    return M_k

print(getBestModel(auto, k=2))

# Check to make sure this is right :-P
getBestModel(auto, k=2) == ('horsepower', 'weight')

Now, once we've found all the $M_k$, we want to return the best model using the *testing* error. So, here's a function that will take in a list of variables and give you the 5-fold CV score. 

In [ ]:
def myscore_cv(df,listofvars, outputvar = 'mpg'):


    if listofvars == 'null model':
        # Predict the mean of the output variable for any input
        test_scores = [] 
        for (train,test) in KFold(5).split(df):
            y_train = df[outputvar].iloc[train]
            y_test = df[outputvar].iloc[test]
            yhat = np.full(y_test.shape[0],np.average(y_train))
            test_scores.append(mean_squared_error(y_test, yhat))
        return np.average(test_scores)
    else:
        X = df[list(listofvars)]
        y = df[outputvar]
        #build linear regression model
        model = LinearRegression()
        
        #use 5-fold CV to evaluate model
        scores = cross_val_score(model, X,y, 
                                scoring='neg_mean_squared_error',
                                cv=5)

        #view mean absolute error
        return np.average(np.absolute(scores))
    

myvars = ('cylinders', 'acceleration')
myscore_cv(auto,myvars)


&#9989; **<font color=red>Do this:</font>** Put it all together. Finish the for loop that runs through $k=0$ to $k=5$, and gets the set of variables giving the best size $k$ model, and then computs the test error for each model. Which model would be returned by the best subset selection method? 


In [ ]:

for k in range(5):
    M_k = None #<-- Use the getBestModel function here to get the list of variables
    print(f"Best {k} variable model: {M_k}")
    M_k_CV = 0 #<-- Use the myscore_cv function here to get the test error
    print(f"CV Score: {M_k_CV:0.3f}")




-----
### Congratulations, we're done!
Initially created by Dr. Liz Munch, adapted by Dr. Mengsen Zhang, Michigan State University

<a rel="license" href="http://creativecommons.org/licenses/by-nc/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-nc/4.0/88x31.png" /></a><br />This work is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-nc/4.0/">Creative Commons Attribution-NonCommercial 4.0 International License</a>.